<a href="https://colab.research.google.com/github/KamilShape/building_style_recognition/blob/main/computer_vision_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np


### Connecting to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Getting Images

In [3]:
styles = ['greek-architecture', 'gothic-architecture', 'baroque-architecture', 'victorian-architecture', 'modern architecture']

In [4]:
def parse_url(content):
  soup = BeautifulSoup(content, 'html.parser')
  imgs = soup.find_all('img', {'class': 'grid-image'})
  return [img['src'] for img in imgs]

In [7]:
url = f'https://www.freeimages.com/search'
def get_page(url,style, page):
  for page in range(1, page + 1):
    url = url + f'/{style}/{page}'
    try:
      response = requests.get(url)
      content = response.content
      return content
    except:
      print('Error! Wrong url.')

In [8]:
def get_image(photo_url, path, file_name, style):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{style}', exist_ok=True)
    img.save(f'{path}/{style}/{file_name}', 'png')

In [9]:
train_path = 'drive/MyDrive/architecture_photos/data/train'

In [10]:
def get_photos(style, page, path):
  content = get_page(style, page)
  images = parse_url(content)
  for image in images:
    if len(image) > 5:
      file_name = image.split('/')[-1]
      get_image(image, train_path, file_name, style)


In [66]:
#DOWNLOADING ALL PHOTOS
for style in styles:
  get_photos(style, 10, train_path)

### Test and train data sets

In [11]:
test_path = 'drive/MyDrive/architecture_photos/data/test'

In [ ]:
os.makedirs(f'{test_path}')

In [12]:
def create_test_set(styles, train_path, test_path, set_size):
  for style in styles:
    files_folder = os.listdir(f'{train_path}/{style}')
    for n in range(0, round(len(files_folder)*set_size)+1):
      files_folder = os.listdir(f'{train_path}/{style}')
      photo_number = np.random.randint(len(files_folder))
      os.makedirs(f'{test_path}/{style}', exist_ok=True)
      os.replace(f'{train_path}/{style}/{files_folder[photo_number]}', f'{test_path}/{style}/{files_folder[photo_number]}')

In [69]:
create_test_set(styles, train_path, test_path, 0.15)

Model 

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras

In [14]:
img_height = 64
img_width = 64
batch_size = 30

In [31]:
train_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.2,
    seed=0,
    subset='training',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

val_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.5,
    seed=0,
    subset='validation',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

Found 244 files belonging to 5 classes.
Using 196 files for training.
Found 244 files belonging to 5 classes.
Using 122 files for validation.


In [35]:
model = Sequential()
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(64, (4,4), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(32, (4,4), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(16, (4,4), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(8, (4,4), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(5, activation='softmax'))

In [36]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [38]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 30,
    batch_size = batch_size)

Epoch 1/30
7/7 [==============================] - 5s 665ms/step - loss: 0.9679 - accuracy: 0.6020 - val_loss: 1.2124 - val_accuracy: 0.5984
Epoch 2/30
7/7 [==============================] - 4s 546ms/step - loss: 0.9242 - accuracy: 0.6531 - val_loss: 1.2065 - val_accuracy: 0.5738
Epoch 3/30
7/7 [==============================] - 4s 567ms/step - loss: 0.8497 - accuracy: 0.7296 - val_loss: 1.1430 - val_accuracy: 0.6148
Epoch 4/30
7/7 [==============================] - 5s 776ms/step - loss: 0.8705 - accuracy: 0.6684 - val_loss: 1.3079 - val_accuracy: 0.5164
Epoch 5/30
7/7 [==============================] - 4s 584ms/step - loss: 0.8176 - accuracy: 0.6684 - val_loss: 1.1515 - val_accuracy: 0.6148
Epoch 6/30
7/7 [==============================] - 4s 532ms/step - loss: 0.7530 - accuracy: 0.7092 - val_loss: 1.2996 - val_accuracy: 0.5738
Epoch 7/30
7/7 [==============================] - 5s 662ms/step - loss: 0.7676 - accuracy: 0.7551 - val_loss: 1.2271 - val_accuracy: 0.5820
Epoch 8/30
7/7 [====